In [3]:
import streamlit_functions as sf
import importlib as imp
import geopandas as gpd
import matplotlib.pyplot as plt
import functions as f
# debugging
import importlib as imp
imp.reload(f)

# graph
import networkx as nx
import h3
from functions import H3_INDEX_RESOLUTION
from shapely.geometry import Point

In [25]:
bikelane_all = gpd.read_parquet('dataset/raw_unprocessed/bikelane_dk.parquet')

In [26]:
bikelane_all.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

### Generating H3 index
Dataset must be in WGS84/EPSG:4326. Takes time, dont run it

In [27]:
def assign_h3_index_to_linestring(gdf, resolution):
    def get_h3_index_of_linestring(line):
        # Use the centroid of the line for indexing
        return h3.geo_to_h3(line.centroid.x, line.centroid.y, resolution)
    
    gdf['h3_index'] = gdf.geometry.apply(get_h3_index_of_linestring)
    return gdf

bikelane_all_h3_indexed = assign_h3_index_to_linestring(bikelane_all, resolution=H3_INDEX_RESOLUTION)

#### checks some index, whenever it is correct or not

 Note: Indexing works only with WGS84

In [29]:
set(bikelane_all_h3_indexed.h3_index.values)

{'8663abb77ffffff',
 '866202dafffffff',
 '866216827ffffff',
 '866212cefffffff',
 '866234097ffffff',
 '866210cefffffff',
 '8662153afffffff',
 '866234487ffffff',
 '8662168c7ffffff',
 '866216b87ffffff',
 '8662334afffffff',
 '866214b87ffffff',
 '86621252fffffff',
 '86621058fffffff',
 '866215787ffffff',
 '866212247ffffff',
 '8662156f7ffffff',
 '86621444fffffff',
 '866210737ffffff',
 '866216217ffffff',
 '86621615fffffff',
 '866236b8fffffff',
 '866202cf7ffffff',
 '8662142dfffffff',
 '866233707ffffff',
 '866230597ffffff',
 '8662a5a97ffffff',
 '86621010fffffff',
 '866216437ffffff',
 '866206747ffffff',
 '8662060afffffff',
 '8662a52c7ffffff',
 '8663a92dfffffff',
 '866234c87ffffff',
 '8662368b7ffffff',
 '8662a535fffffff',
 '8662164a7ffffff',
 '86620628fffffff',
 '86621551fffffff',
 '8662148c7ffffff',
 '8662363afffffff',
 '866236667ffffff',
 '866213977ffffff',
 '86638b6dfffffff',
 '8662148e7ffffff',
 '866236cc7ffffff',
 '86623611fffffff',
 '8662146cfffffff',
 '866210917ffffff',
 '86621571fffffff',


In [9]:
f.h3_index_visualizer("86621444fffffff")

In [31]:
 # save
bikelane_all_h3_indexed.to_parquet('dataset/raw_unprocessed/bikelane_dk_WGS84_h3_indexed.parquet')

### find closest bikelines based on the h3 index

In [11]:
bikelane_all = gpd.read_parquet('dataset/raw_unprocessed/bikelane_dk_WGS84_h3_indexed.parquet')

In [5]:
coordinates1 = [[11.581726, 55.606109], [11.699829, 55.592143]] # sjeland
coordinates2 = [[12.473907, 55.717571], [12.561798, 55.695132]] # in Copenhagen
coordinates3 = [[12.200317, 55.864524], [11.881714, 55.523967]] # big distance

In [57]:
imp.reload(f)
import functions as f
f.get_h3_index_from_point(coordinates1[0])

'861f04b47ffffff'

In [62]:
# Assuming 'bikelane_all' is your GeoDataFrame of bike lanes
# and 'coordinates2' are your start and end points in Copenhagen

# Convert points to Shapely Points
start_point = Point(coordinates2[0])
end_point = Point(coordinates2[1])


# Function to find the closest LineString to a point, considering H3 index
def find_closest_objects(point, gdf):
    point_h3_index = f.get_h3_index_from_point(point)
    # Filter linestrings by H3 index
    filtered_gdf = gdf[gdf['h3_index'] == point_h3_index]
    if len(filtered_gdf) == 0:
        # If no lines are found in the same H3 index, search in the neighboring indexes
        neighbors = h3.k_ring(point_h3_index, 1)
        filtered_gdf = gdf[gdf['h3_index'].isin(neighbors)]
    
    min_dist = float('inf')
    closest_line = None
    for row in filtered_gdf.itertuples():
        # Calculate the distance from the point to the current line
        dist = point.distance(row.geometry)
        if dist < min_dist:
            min_dist = dist
            closest_line = row.geometry
            closest_index = row.Index  # itertuples gives us the index in the 'Index' attribute
            closest_line_point = f.closest_coordinate_on_linestring(point, closest_line)
    return closest_line, closest_index, closest_line_point, filtered_gdf

# Find closest LineStrings to start and end points
print("find_closest_linestring")
closest_line_start, closest_index_start, closest_line_point_start, filtered_gdf_1 = find_closest_objects(start_point, bikelane_all)
closest_line_end, closest_index_end, closest_line_point_end, filtered_gdf_2 = find_closest_objects(end_point, bikelane_all)
print("find_closest_linestring end")

find_closest_linestring
find_closest_linestring end


In [63]:
print(closest_line_start, closest_index_start, closest_line_point_start)
print(closest_line_end, closest_index_end, closest_line_point_end)

LINESTRING (12.4777437 55.7171138, 12.4768664 55.7174166, 12.4766548 55.7174636, 12.4764077 55.7174804, 12.4761584 55.7174662, 12.4751326 55.7172594, 12.4732513 55.7168712, 12.4730847 55.7168092, 12.4727463 55.716573, 12.4725479 55.7165042, 12.4715788 55.716314, 12.4698784 55.7159803, 12.4697847 55.7159517, 12.4694657 55.7157058, 12.4693875 55.7156994, 12.4692411 55.7156978) (283279508, 423158236, 0) POINT (12.4732513 55.7168712)
LINESTRING (12.5614365 55.6958225, 12.5613083 55.6947673) (269598178, 7773050, 0) POINT (12.5613083 55.6947673)


#### THIS IS IT

In [46]:
imp.reload(f)
import functions as f
t = f.filter_geopandas_by_u_v_index(closest_index_start[0], closest_index_start[1], bikelane_all)

283279508 423158236


d:\ITU\Geospatial Data Science\PROJECT\functions.py:103: PerformanceWarning: indexing past lexsort depth may impact performance.
  filtered_gdf = gdf[(gdf['u'] == u_value) | (gdf['v'] == v_value)]


In [22]:
# Find closest nodes to start and end points
start_node, _ = nearest_points(start_point, closest_line_start)
end_node, _ = nearest_points(end_point, closest_line_end)
print("start_node", start_node)
print("end_node", end_node)

start_node POINT (12.473907 55.717571)
end_node POINT (12.561798 55.695132)


In [44]:
G_edges = nx.Graph()

for row in bikelane_all.itertuples():
    # Access the MultiIndex values (u, v, key) from the Index
    u, v, _ = row.Index

    # Add nodes
    G_edges.add_node(u)
    G_edges.add_node(v)

    # Add edge
    G_edges.add_edge(u, v)


In [45]:
# Write to GraphML
nx.write_graphml(G_edges, 'graph.graphml')

In [72]:
bikelane_all.total_bounds

array([ 441698.70518066, 6072912.09409883,  728400.29103142,
       6402147.68476988])

### Without indexing the below code is super slow since it loops through the whole database to find the closest bikelane

### H3 indexing

In [77]:
bounds = bikelane_all.geometry.total_bounds
print(bounds)

from shapely.geometry import box
import geopandas as gpd

# Create a Polygon from bounds
bound_box = box(*bounds)

# Convert to a GeoDataFrame
gdf_bounds = gpd.GeoDataFrame(geometry=[bound_box], crs=bikelane_all.crs)

print(gdf_bounds)

# check these coordinates are inside the box 



[ 441698.70518066 6072912.09409883  728400.29103142 6402147.68476988]
                                            geometry
0  POLYGON ((728400.291 6072912.094, 728400.291 6...


In [84]:
gdf_bounds.explore()